# Study Summary Report
> Author: Clayton Herbst

## Business Needs Statement
CRO need to be able to review the state of a study on demand, following the progression of study participants over time using key metrics captured within the study.

## Task Description
Generate a report that reflects the current data within the study, collating all ECG, Vital Signs, PK data on a per tab basis ensuring the links to study participants, forms, visits and timepoints remain. Data must accurately reflect the data stored in the study database.

In [1]:
from openpyxl import Workbook, load_workbook
import json
import pprint
import datetime
import os
import re
from matplotlib import pyplot

%matplotlib inline

## Setup Output Folder

Create the __output folder__ in which all output files will be placed.

In [2]:
# Set the output folder location
outputfolder = './test/output/coh2'

In [3]:
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder, 0o755) # owner execution 

View contents of folder:

In [4]:
%ls -l ./test

total 20736
-rw-r--r--@ 1 herbsca  staff  10606921 18 May 11:31 browse_data.xlsx
-rw-r--r--@ 1 herbsca  staff       997 18 May 16:15 config.json
drwxr-xr-x  3 herbsca  staff        96 18 May 10:35 output/
-rw-r--r--@ 1 herbsca  staff       165 18 May 11:31 ~$browse_data.xlsx


## Define Utility Functions
These functions generate the format of the output file. They are supplied with a _dictionary variable_ (`contents`) and then convert this information structure into the output file format (csv -> columns, json -> key-value pairs etc).

In [5]:
def writeFile(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(pprint.pformat(contents))
    print('Text file write complete.')

def writeJSON(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(json.dumps(contents))
    print('JSON file write complete.')

def writeErrorCSV(filename: str, contents: object):
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        fieldnames = ['subjectid', 'type', 'collection', 'visit', 'panel', 'sourceDate', 'sourceTime', 'comparisonDate', 'comparisonTime', 'message']
        writer = csv.DictWriter(outputfile, fieldnames=fieldnames)
        writer.writeheader() # place headers in csv file.
        for key in contents:
            if key == 'errors' or  key  == 'total':
                    continue
            for data in contents[key]:
                # cycle through list of errors
                sourcedate = '-'
                sourcetime = '-'
                comparisondate = '-'
                comparisontime = '-'
                msg =  ''
                if 'date' in data and data['date']['error']:
                    sourcedate = data['date']['source']
                    comparisondate = data['date']['comparison']
                if 'time' in data and data['time']['error']:
                    sourcetime = data['time']['source']
                    comparisontime = data['time']['comparison']
                if 'msg' in data:
                    msg = data['msg']

                row = {
                    'subjectid': data['subject'],
                    'visit': data['visit'],
                    'type': data['type'],
                    'collection': data['collection'],
                    'panel': data['panel'],
                    'sourceDate': sourcedate,
                    'sourceTime': sourcetime,
                    'comparisonDate': comparisondate,
                    'comparisonTime': comparisontime,
                    'message': msg
                }
                writer.writerow(row) # write to file
    print('CSV file write complete.')

## Excel File Reading Class

Python class created for the application to read from the supplied Medrio __browse data__ report.

### About
Uses the _openpyxl_ external package (make sure this is installed in your environment!). In order to handle the large file sizes the optimised _read-only_ mode is used. This limits the extent of operations that can be performed with the file (cell selection) however suits this application perfectly.

In [6]:
class FileReader:
    def __init__(self, filename):
        '''
        @param filename - location of xlsx workbook file (Medrio Browse Data File)
        '''
        if not os.path.exists(filename):
            raise Exception('IOERROR: Could not find the input file')
        self.filename = filename
        self.wb = load_workbook(filename=filename, read_only=True) # instance workbook

    def getSheetnames(self):
        return self.wb.sheetnames

    def getWorksheet(self, sheetname=None):
        '''
        @raise KeyError - if worksheet does not exist
        '''
        if sheetname is None:
            return self.wb.active
        else:
            return self.wb[sheetname]


    def getSheetHeaders(self, sheetname=None):
        '''
        @raise KeyError - if worksheet does not exist
        @param sheetname - name of the sheet to retrieve headers from
        '''
        ws = self.getWorksheet(sheetname);
        toprow = ws.iter_rows(max_row=1) # tuple containing tuple's of Cell objects
        # print(toprow)
        headers = []
        for row in toprow:
            for cell in row:
                # print(cell.value)
                if cell.value == None:
                    break
                else:
                    headers.append(cell.value)
            break;
        return headers

### Test XLSX Reading Class

In [7]:
inputfilepath = "./test/browse_data.xlsx"

filereader = FileReader(inputfilepath)
ws = filereader.getWorksheet()

print(filereader.getSheetnames())
print(ws.calculate_dimension())

i = 0
'''for row in ws.rows:
    i += 1
    if i < 10:
        print(row[4].value)
    elif i == 1946:
        print(row[4].value)
        break
'''
print(' --- ')
filereader.getSheetHeaders()

['Data']
A1:BKX3430
 --- 


['Medrio ID',
 'Group',
 'Subject ID',
 'Visit',
 'Form',
 'PKPD_Rec_C',
 'PKPD_NoRecSpe_C',
 'PKPD_Date_C',
 'PKPD_Time_C',
 'PKPD_DatTim_calc',
 'SBC_SampCollScng_C',
 'SBC_IfNoCommScng_C',
 'SBC_DatScr_C',
 'SBC_TimScr_C',
 'UColl_Rec_C',
 'UColl_RecNoSpe_C',
 'UColl_Dat_C',
 'UColl_TimPotOut_C',
 'ECGTrpRec_C',
 'ECGTrpNoSpec_C',
 'ECGTrpDat_C',
 'ECGTrpMeasTim1_C',
 'ECGTrp_Heart1a_C',
 'ECGTrp_RR1_C',
 'ECGTrp_PR1_C',
 'ECGTrp_QRSD1_C',
 'ECGTrp_QT1_C',
 'ECGTrp_QTcF1_C',
 'ECGTrpRPRev1_C',
 'ECGTrpRPRevComm1_C',
 'ECGTrpMeasTim2_C',
 'ECGTrp_Heart2a_C',
 'ECGTrp_RR2_C',
 'ECGTrp_PR2_C',
 'ECGTrp_QRSD2_C',
 'ECGTrp_QT2_C',
 'ECGTrp_QTcF2_C',
 'ECGTrpRPRev2_C',
 'ECGTrpRPRevComm2_C',
 'ECGTrpMeasTim3_C',
 'ECGTrp_Heart3a_C',
 'ECGTrp_RR3_C',
 'ECGTrp_PR3_C',
 'ECGTrp_QRSD3_C',
 'ECGTrp_QT3_C',
 'ECGTrp_QTcF3_C',
 'ECGTrpRPRev3_C',
 'ECGTrpRPRevComm3_C',
 'ECGRep_C',
 'ECGRepReas',
 'ECGTrpMeasTim1_R1_C',
 'ECGTrpHeart1a_R1_C',
 'ECGTrp_RR1_R1_C',
 'ECGTrp_PR1_R1_C',
 'ECGTrp_QRSD1

## Excel File Writing Class

Python class to write to file and excel file, with tabs for various key forms and bulk data.

In [8]:
class FileWriterXL:
    def __init__(self, name, fieldnames):
        dirname = os.path.dirname(name)
        if not os.path.exists(dirname):
            print(f'Creating Directory: {dirname}')
            os.mkdir(dirname)
        self.filename = f'{name}.xlsx'
        self.headers = FileWriterXL.setHeaders(fieldnames)
        self.wb = Workbook() # create new workbook
        self.ws = self.wb.active
        self.setWorksheet() # define the headers and worksheet properties

    def setHeaders(fieldnames):
        '''
        @param fieldnames - iterable list of fieldnames
        '''
        headers = {}
        for itx, fieldname in enumerate(fieldnames):
            headers[fieldname] = itx+1
        return headers

    def setWorksheet(self):
        self.ws.title = 'DATA'
        self.ws.sheet_properties.tabColor = 'FF69B4'

        for header, pos in self.headers.items():
            self.ws.cell(row=1, column=pos, value=header)
        self.currentRow = 2 # set the last completed row

    def bulkWrite(self, entries):
        '''
        @param entries - array of dictionary entries with key's containing the col headers
        '''
        for entry in entries:
            for key, value in entry.items():
                self.ws.cell(row=self.currentRow, column=self.headers[key], value=value)
            self.currentRow += 1 # increment row
        return

    def bulkWriteSheet(self, sheetname, entries):
        '''
        @param entries - array of dictionary entries with key's containing the col headers
        '''

        ws = self.wb.create_sheet(title=sheetname)
        currentRow = 1 # set the current row as the header
        # Write the header
        for key, value in self.headers.items():
            ws.cell(row=currentRow, column=value, value=key)
        currentRow += 1

        for entry in entries:
            for key, value in entry.items():
                ws.cell(row=currentRow, column=self.headers[key], value=value)
            currentRow += 1 # increment row
        return

    def singleWrite(self, entry):
        '''
        @param entry - dictionary entry
        '''
        for key, value in entry.items():
            self.ws.cell(row=self.currentRow, column=self.headers[key], value=value)
        self.currentRow += 1 # increment row
        return

    def printHeaders(self):
        print(self.headers.keys())

    def getFilename(self):
        return self.filename

    def close(self):
        '''
        Saves the excel file to file and then closes the file properly.
        '''
        return self.wb.save(self.filename)

## Process Input File Class

In [43]:
class ProcessData:
    
    def __init__(self, variables, form_type_map):
        if headers is None or form_type_map is None:
            raise Exception('Process data class \'None\' paramaters')
        self.variables = variables # headers of the input file
        self.typemap: dict = form_type_map
        self.constant_headers_map = {'Subject ID': 2, 'Form Name': 4, 'Group': 1, 'Visit': 3}

    def process_from_config(self, row, formtype: str):
        '''
        Process row of data using config file specified mapping
        @Return - dict containing all collected data fields
        '''
        if formtype is None:
            raise Exception('Error with typemap for process')
    
        typemap = self.typemap.get(formtype, None) # get the config file map
        if typemap is None or typemap.get('_colregex', None) is None:
            print(f"Warning: typemap not identified {formtype}")
            return None
        
        medrio_order = typemap.get("_medrio_order", None)
        if medrio_order is None:
            raise Exception("Medrio order must be specified in the config file.")
            return None
        
        colregex = typemap.get('_colregex', None)
        if colregex is None: # no regex match
            raise Exception("Typemap has no column regex / variable regex defined")
        
        # -- PARSE DATA --
        row_data = {} # dictionary of data
        col = 0 # column index of xlsx cells
        medrio_index = 0 # keep track of form looking order
        for cell in row:
            col += 1 # increment column
            # cycle through all cells in row
            if cell.value is None or cell.value == '':
                continue # skip empty cell
            
            variable = self.variables[col-1] # medrio variable name
            
            # Follow order
            colname = medrio_order[medrio_index] # get the expected data point
            print(colname)
            searchstr = colregex.get(colname, None)
            print(searchstr)
            if searchstr is None: # no regex match
                continue
            if re.search(searchstr, variable, flags=re.I) is not None: # mapped to variable
                if row_data.get(colname, None) is None: # variable has not already been populated
                    medrio_index = (medrio_index + 1)%len(medrio_order) # increment next expected variable
                    row_data[colname] = cell.value # set value data point
                else: # data point already populated
                    print('Warning: data could be overwritten')
            
            
            '''for colname, searchstr in typemap.get('_colregex', None).items():
                if(re.search(searchstr, variable, flags=re.I) is not None): # mapped to variable
                    if row_data[trip_coll].get(colname, None) is None:
                        medrio_index += 0 # increment next expected variable
                        row_data[colname] = cell.value # set value data point
                    else: # data point already populated
                        print('Data is being overwritten')
            '''
        return row_data
    
    def process_triplicate_from_config(self, row, formtype: str):
        '''
        Need to keep track of what triplicate collection it is.
        '''
        return None
    
    def process_generalised_cells(self, row):
        '''
        Process the column headers that are constant
        '''
        row_dict = dict()
        for header, row_index in self.constant_headers_map.items():
            row_dict[header] = row[row_index].value # extract value of cell
        return row_dict;
        
        
    def generate_form_type(self, formname=None):
        '''
        Generate the identifier for the form (group the forms for output)
        
        @Returns - None if not type map could be made.
        '''
        if formname is None:
            raise Exception('Formname cannot be None')
        formtype = None # form type identifier
        triplicate = False
        for t, typemap in self.typemap.items():
            print(t)
            if typemap.get('_formregex', None) is not None and re.search(typemap.get('_formregex', None), formname, flags=re.I) is not None: # there was a match
                formtype = t
                if typemap.get('_triplicateregex', None) is not None and re.search(typemap.get('_triplicateregex', None), formname, flags=re.I) is not None:
                    triplicate = True
                break
            
        return (formtype, triplicate)
        
        

## Parse Config File

__EDIT HERE:__

In [44]:
config_file_path = './test/config.json'

Parse config file and establish relationships.

In [45]:
config_dict = dict()

with open(config_file_path) as jsonfile:
    config_dict = json.load(jsonfile)

pprint.pp(config_dict)

{'tabs': ['ECG', 'Vital Signs'],
 'map': {'ECG': {'_formregex': 'ecg',
                 '_col': ['date',
                          'time',
                          'heart',
                          'rr',
                          'pr',
                          'qrsd',
                          'qt',
                          'qtcf',
                          'asssessment'],
                 '_medrio_order': ['date',
                                   'time',
                                   'heart',
                                   'rr',
                                   'pr',
                                   'qrsd',
                                   'qt',
                                   'qtcf',
                                   'assessment'],
                 '_colregex': {'date': 'dat',
                               'time': 'tim',
                               'heart': 'heart',
                               'rr': 'rr',
                               'pr': 'pr',
    

## Process Workbook

In [46]:
input_file_path = './test/browse_data.xlsx'

Read the xlsx file and create a searchable dictionary variable.

In [47]:
filereader = FileReader(input_file_path)
headers = filereader.getSheetHeaders()
ws = filereader.getWorksheet()
pd = ProcessData(headers, config_dict.get('map', None)) # pd = processdata

row_id = 0
for row in ws.rows:
    data = dict()
    row_id += 1 # increment the row
    if row_id == 1:
        continue # skip the xlsx headers
    if row_id > 50:
        break 
    general = pd.process_generalised_cells(row)
    print(general.get('Form Name', None))
    holder = pd.generate_form_type(general.get('Form Name', None)) # get the form identifier
    
    formtype = holder[0]
    is_triplicate = holder[1]
    
    print(formtype)
        
    if formtype is None:
        continue
    
    if is_triplicate: # is triplicate
        continue
        data = pd.process_triplicate_from_config(row, formtype)
        if data is None:
            continue
    else: # not triplicate
        data = pd.process_from_config(row, formtype)
        print(data)
        if data is None:
            continue
    
    pprint.pp(general)
    print(data)
    pprint.pp({**general, **data})

Neurological Exam
ECG
Vital Signs
None
Neurological Exam
ECG
Vital Signs
None
Complete Physical Exam
ECG
Vital Signs
None
Complete Physical Exam
ECG
Vital Signs
None
1.5HR Neurological Exam (Second Dose)
ECG
Vital Signs
None
Neurological Exam
ECG
Vital Signs
None
Complete Physical Exam
ECG
Vital Signs
None
1HR PK Blood Collection (Second Dose)
ECG
Vital Signs
None
Neurological Exam
ECG
Vital Signs
None
Complete Physical Exam
ECG
Vital Signs
None
0.5HR Neurological Exam (First Dose)
ECG
Vital Signs
None
Complete Physical Exam
ECG
Vital Signs
None
Complete Physical Exam
ECG
Vital Signs
None
30MIN Pre-dose Vital Signs
ECG
Vital Signs
Vital Signs
date
dat
date
dat
date
dat
date
dat
date
dat
date
dat
date
dat
time
meas
systolic
syst
diastolic
dias
heart
heart
resp
resp
temp
temp
assessment
rev
date
dat
{'date': '04/17/2020', 'time': '06:55', 'systolic': 120, 'diastolic': 80, 'heart': 59, 'resp': 17, 'temp': 36.6, 'assessment': 'No'}
{'Subject ID': 'S003-11001',
 'Form Name': '30MIN Pre-dose